This notebook was prepared by [SentinelWarren](https://github.com/sentinelwarren). Source and license info is on [GitHub](https://github.com/sentinelwarren/patterns-in-langs).

# Singleton Pattern

## Description

Is a way to provide one and only one object of a particular type. The Singleton class can have only one instance, and provides global access to that one instance.

Due to its instantiation restrictions of a class to one object, this pattern is useful when you need one object to coordinate actions for the system.

The basic idea is that only one instance of a particular class, doing a job, is created for the needs of the program. To ensure that this works, we need mechanisms that prevent the instantiation of the class more than once and also prevent cloning.

## Singleton in the wild

Singleton pattern can be used to implement projects that do not affect the execution of code, hence its acceptable to use global state. The kinds of projects are;

* Logging
* Caching
* Load balancing
* Route mapping

In all these cases,information flows in one direction, and the singleton instance itself is immutable (it does not change).
No part of the program attempts to make a change in the singleton, and as such there is no danger of one part of a project interfering with another part of the project because of the shared state.

### Example projects

* Plone CMS

## Pros & Cons

### Pros

* Access to global state, making a Singleton instance to be immutable.

### Cons | Drawbacks

* Possibility of coupling your code to itself in very unfortunate ways, causing unexpected results in completely unrelated piece of code.

## Use cases

* Maintaining a global state of a program

* Controlling concurrent access to a shared resource. For example, the class managing the connection to a database.

* A service or resource that is transversal in the sense that it can be accessed from different parts of the application or by different users and do its work. For example, the class at the core of the logging system or utility.

## Implementation

* Using metaclass and its type, having predefined the `Singleton` metaclass.
* As required, the metaclass's `__call__()` method holds the code that ensures that only one instance of the class can be created.

### Test cases

#### Unit

* `len(SingletonClass().instances) == 0`
* `SingletonClass() == SingletonClass()`
* `SingletonClass().attributes == SingletonClass().attributes`
* `len(SingletonClass().instances) == 1`
* `len(SingletonClass().instances) >= 1`

#### Functional

* `Singleton().instances['invalid'] raise KeyError`

In [2]:
%%writefile singleton.py
from typing import Dict


class Singleton(type):
    _instances: Dict[str, object] = dict()

    def __call__(cls, *args: object, **kwargs: object) -> object:
        if str(cls) not in cls._instances:
            cls._instances[str(cls)] = super(Singleton,
            cls).__call__(*args, **kwargs)

        return cls._instances[str(cls)]


class FooTone(metaclass=Singleton):
    def __init__(self, arg: str) -> None:
        self.attribute = arg

    def __repr__(self) -> str:
        return (
            f"<class {self.__class__.__name__}("
            f"{self.attribute!r}) "
            f"at {hex(id(self))}>"
        )

Overwriting singleton.py


## Tests

Using `pytest`

In [1]:
%%writefile ../../tests/unit/test_singleton.py
from pypatterns import Singleton, FooTone, Logger


foo = "<class 'patterns.singleton.singleton.FooTone'>"
logger = "<class 'patterns.singleton.logger.Logger'>"

def instantiate_foo(arg):
    return FooTone(arg)

def instantiate_logger(arg):
    return Logger(arg)

def delete_instance(name):
    if name in Singleton._instances.keys():
        del Singleton._instances[name]

def test_global_state():
    foo1 = instantiate_foo("some_attr")
    foo2 = instantiate_foo("diff_attr")
    assert foo1 == foo2
    assert foo1.attribute == foo2.attribute

    foo2.attribute = "another_attr"
    assert foo1.attribute == foo2.attribute

def test_empty_instances():
    Singleton._instances.clear()
    
    assert len(Singleton._instances) == 0
    assert logger, foo not in Singleton._instances

def test_all_instances():
    if logger not in Singleton._instances:
        instantiate_logger("file.log")
    
    if foo not in Singleton._instances:
        instantiate_foo("some_attr")
    
    assert len(Singleton._instances) == 2
    assert foo, logger in Singleton._instances

def test_logger_instance():
    delete_instance(foo)
    
    assert len(Singleton._instances) == 1
    assert logger in Singleton._instances

def test_foo_instance():
    instantiate_foo("some_attr")
    delete_instance(logger)
    
    assert len(Singleton._instances) == 1
    assert foo in Singleton._instances

Overwriting ../../tests/unit/test_singleton.py


In [2]:
%%writefile ../../tests/func/test_singleton_exceptions.py
from pypatterns import Singleton
from pytest import raises

def test_instances_raises():
    with raises(KeyError):
        Singleton._instances['invalid']

Overwriting ../../tests/func/test_singleton_exceptions.py


In [1]:
%run -m pytest ../../tests/unit/test_singleton.py ../../tests/func/test_singleton_exceptions.py

......                                                                   [100%]
6 passed in 0.06s


# Project implementation

A simple project implementation using the Singleton pattern.

## `logger.py`

A simple logging module that keeps track of the `.log` file state, writes specific logging messages to the log file accordingly, and reads the contents of the log file.

In [3]:
%%writefile logger.py
from pypatterns import Singleton
from typing import Any


class Logger(metaclass=Singleton):
    def __init__(self, file_name: str) -> None:
        self.file_name = file_name

    def _write_log(self, level: str, msg: Any) -> None:
        with open(self.file_name, "a") as log_file:
            log_file.write(f"[{level}] {msg}\n")

    def _read_log(self) -> str:
        with open(self.file_name, "r") as log_file:
            return log_file.read()

    def critical(self, msg: Any) -> None:
        self._write_log('CRITICAL', msg)
    
    def error(self, msg: Any) -> None:
        self._write_log('ERROR', msg)

    def warn(self, msg: Any) -> None:
        self._write_log('WARNING', msg)

    def info(self, msg: Any) -> None:
        self._write_log('INFO', msg)

    def debug(self, msg: Any) -> None:
        self._write_log('DEBUG', msg)

    def print_log(self) -> None:
        print(self._read_log())

    def __repr__(self) -> str:
        return (
            f"<class {self.__class__.__name__}("
            f"{self.file_name!r}) "
            f"at {hex(id(self))}>"
        )



if __name__ == '__main__':
    logger = Logger("class_logger.log")
    logger.warn('AE-35 hardware failure predicted!')
    logger.print_log()
    

Overwriting logger.py


## Functional Test

Using `pytest`

In [4]:
%%writefile ../../tests/func/test_logger.py
from pypatterns import Logger


def test_methods_return(tmpdir_factory):
    tmp = tmpdir_factory.mktemp("func")
    logger = Logger(tmp.join("logger.log"))
    expected = None

    assert logger.critical("This is a critical message") == expected
    assert logger.error("This is an error message") == expected
    assert logger.warn("This is a warning message") == expected
    assert logger.info("This is an info message") == expected
    assert logger.debug("This is a debugging message") == expected
    assert logger.print_log() == expected

def test_log_output():
    expected_log = (
        f"[CRITICAL] This is a critical message\n"
        f"[ERROR] This is an error message\n"
        f"[WARNING] This is a warning message\n"
        f"[INFO] This is an info message\n"
        f"[DEBUG] This is a debugging message\n"
    )
    assert Logger("logger.log")._read_log() == expected_log

Overwriting ../../tests/func/test_logger.py


In [1]:
%run -m pytest ../../tests/func/test_logger.py

..                                                                       [100%]
2 passed in 0.03s
